In [1]:
import copy
import pickle
import re
from typing import List
import tqdm

import numpy as np

In [2]:
import openff.interchange
import openff.toolkit

from openff.interchange.components.interchange import Interchange
from openff.toolkit.topology import Topology
from openff.toolkit.typing.engines.smirnoff import ForceField

import openmm
import pandas as pd

In [3]:
print(openff.interchange.__version__)

v0.1.4


In [4]:
print(openff.toolkit.__version__)

0.10.6


In [5]:
from interchange_regression_utilities.models import (
    ComparisonSettings,
    ExpectedValueChange,
    model_from_file,
    TopologyDefinition,
)

In [6]:
models = model_from_file(List[TopologyDefinition], "input-topologies.json")

In [8]:
topologies = []
failed_smiles = []
smiles_pattern = "SMILES '([A-Za-z0-9=\(\)\[\]]+)'"
for model in tqdm.tqdm(models):
    try:
        topologies.append(model.to_topology())
    except ValueError as e:
        if "No registered toolkits can provide the capability" in str(e):
            smiles = re.search(smiles_pattern, str(e)).group(1)
            failed_smiles.append(smiles)

 51%|█████████████████████▉                     | 26/51 [08:18<09:44, 23.39s/it]Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 4, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 3, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 5, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 15, aromatic: False, chiral: False

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 4, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 3, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 5, aromatic: True, chiral:

In [9]:
len(failed_smiles)

0

In [10]:
topology_file = "openff_topologies_v0.10.pkl"
with open(topology_file, "wb") as f:
    pickle.dump(topologies, f)
# with open(topology_file, "rb") as f:
#     topologies = pickle.load(f)

In [14]:
force_field = ForceField("openff-2.0.0.offxml")

In [15]:
interchanges = [
    Interchange.from_smirnoff(force_field, topology)
    for topology in tqdm.tqdm(topologies)
]

interchange_systems = [
    ichange.to_openmm(combine_nonbonded_forces=True)
    for ichange in interchanges
]

 51%|█████████████████████▉                     | 26/51 [01:22<01:32,  3.69s/it]Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: N1x, idx: 4, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: C2x, idx: 3, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: C3x, idx: 5, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: C9x, idx: 15, aromatic: False, chiral: False

 67%|████████████████████████████▋              | 34/51 [01:44<00:47,  2.80s/it]Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: N2x, idx: 7, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: C6x, idx: 6, aromatic: True, chiral: 

In [17]:
# toolkit_systems = [
#     force_field.create_openmm_system(topology, use_interchange=False)
#     for topology in tqdm.tqdm(topologies)
# ]

In [18]:
def get_charges(
    system,
    model_index: int = 0,
    version: str = "interchange",
) -> pd.DataFrame:
    force = [
        force for force in system.getForces()
        if "NonbondedForce" in str(type(force))
    ][0]
    
    name = models[model_index].name
    data = {"particle_index": [], "charge": []}
    for i in range(force.getNumParticles()):
        charge, *_ = force.getParticleParameters(i)
        data["particle_index"].append(i)
        data["charge"].append(charge._value)
    
    df = pd.DataFrame(data)
    df["name"] = name
    df["version"] = version
    return df

In [20]:
all_charge_dfs = []

for i, (ichange) in tqdm.tqdm(
    list(enumerate((interchange_systems)))
):
    model = models[i]
    
    name = model.name
    ixml = openmm.XmlSerializer.serialize(ichange)
#     txml = openmm.XmlSerializer.serialize(toolkit)
    
    ichange_name = f"interchange-{openff.interchange.__version__}"
#     toolkit_name = f"toolkit-{openff.toolkit.__version__}"
    ifile = f"xmls/{name}_{ichange_name}.xml"
#     tfile = f"xmls/{name}_{toolkit_name}.xml"
    
    with open(ifile, "w") as f:
        f.write(ixml)
#     with open(tfile, "w") as f:
#         f.write(txml)
        
    idf = get_charges(ichange, model_index=i, version=ichange_name)
#     tdf = get_charges(toolkit, model_index=i, version=toolkit_name)
    
    model_df = copy.deepcopy(idf)
    model_df["version"] = "original_topology_v0.10"
    model_df["charge"] = [
        charge
        for molecule in topologies[i].topology_molecules
        for charge in np.array(molecule.reference_molecule.partial_charges)
    ]
    
#     topology2 = model.to_topology()
#     model_df2 = pd.DataFrame(tdf)
#     model_df2["version"] = "new_topology"
#     model_df2["charge"] = [
#         charge._value
#         for molecule in topologies[i].topology_molecules
#         for charge in molecule.partial_charges
#     ]
    
    
    all_charge_dfs.extend(
        [idf, model_df,
#          model_df2,
    ])

charge_df = pd.concat(all_charge_dfs)


100%|██████████████████████████████████████████| 51/51 [00:00<00:00, 199.85it/s]


In [21]:
charge_df.version.unique()

array(['interchange-v0.1.4', 'original_topology_v0.10'], dtype=object)

In [25]:
charge_df.to_csv("charges_v010.csv")